In [1]:
import pandas as pd
import re

from collections import Counter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import wordnet

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
casual = pd.read_csv('./Data/casual_convo.csv')

/Users/johnnybex/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8,9,72,73) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
political = pd.read_csv('./Data/politics.csv')

/Users/johnnybex/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,1,4,6,8,9,10,11,12,16,18,19,20,21,22,24,26,27,28,30,31,33,34,38,40,46,47,52,53,57,60,65,68,70,71,74,95,96,97,98) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
casual.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,author_premium,awarders,can_mod_post,contest_mode,created_utc,domain,full_link,gildings,id,is_crosspostable,is_meta,is_original_content,is_reddit_media_domain,is_robot_indexable,is_self,is_video,link_flair_background_color,link_flair_richtext,link_flair_template_id,link_flair_text,link_flair_text_color,link_flair_type,locked,media_only,no_follow,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,pinned,pwls,retrieved_on,score,selftext,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_subscribers,subreddit_type,thumbnail,title,total_awards_received,treatment_tags,upvote_ratio,url,whitelist_status,wls,link_flair_css_class,removed_by_category,poll_data,author_flair_background_color,author_flair_text_color,banned_by,edited,author_flair_template_id,post_hint,preview,author_cakeday,distinguished,collections,discussion_type,suggested_sort,gilded
0,[],False,V-Jessica-V,NaN,[],NaN,text,t2_9v0jl1t6,False,False,[],False,False,1611880538,self.CasualConversation,https://www.reddit.com/r/CasualConversation/co...,{},l7e0yt,True,False,False,False,True,True,False,#638be4,"[{'a': ':chat:', 'e': 'emoji', 'u': 'https://e...",58904be0-3089-11e9-b6c6-0e12b0807136,:chat: Just Chatting,light,richtext,False,False,False,0,0,False,all_ads,/r/CasualConversation/comments/l7e0yt/f_21_loo...,False,6,1611880549,1,(Name): Jessica\n(Age): 21\n(Height): 5’10 (So...,True,False,False,CasualConversation,t5_323oy,1440677,public,self,"F, 21. Looking for people to chat to :D",0,[],1.0,https://www.reddit.com/r/CasualConversation/co...,all_ads,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,shaggy1452,NaN,[],NaN,text,t2_178m2o,False,False,[],False,False,1611880297,self.CasualConversation,https://www.reddit.com/r/CasualConversation/co...,{},l7dxgq,True,False,False,False,True,True,False,#638be4,"[{'a': ':chat:', 'e': 'emoji', 'u': 'https://e...",58904be0-3089-11e9-b6c6-0e12b0807136,:chat: Just Chatting,light,richtext,False,False,True,0,0,False,all_ads,/r/CasualConversation/comments/l7dxgq/did_my_f...,False,6,1611880308,1,I haven’t done contact spirts (or any sports b...,True,False,False,CasualConversation,t5_323oy,1440674,public,self,Did my first Brazilian Jiujitsu class today,0,[],1.0,https://www.reddit.com/r/CasualConversation/co...,all_ads,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,old_lady_young_body,NaN,[],NaN,text,t2_7bvilq6l,False,False,[],False,False,1611879395,self.CasualConversation,https://www.reddit.com/r/CasualConversation/co...,{},l7dku0,True,False,False,False,True,True,False,NaN,[],NaN,NaN,dark,text,False,False,True,0,0,False,all_ads,/r/CasualConversation/comments/l7dku0/cleaning...,False,6,1611879405,1,Guyssssssssssssss maybe I'm late to the game o...,True,False,False,CasualConversation,t5_323oy,1440656,public,self,Cleaning my converse,0,[],1.0,https://www.reddit.com/r/CasualConversation/co...,all_ads,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,randomusername_741,NaN,[],NaN,text,t2_8ppfhsf5,False,False,[],False,False,1611879042,self.CasualConversation,https://www.reddit.com/r/CasualConversation/co...,{},l7dfv6,True,False,False,False,True,True,False,#638be4,"[{'a': ':chat:', 'e': 'emoji', 'u': 'https://e...",58904be0-3089-11e9-b6c6-0e12b0807136,:chat: Just Chatting,light,richtext,False,False,False,5,0,False,all_ads,/r/CasualConversation/comments/l7dfv6/do_you_t...,False,6,1611879052,1,\n\n[View Poll](https://www.reddit.com/poll/l7...,True,False,False,CasualConversation,t5_323oy,1440646,public,self,Do you think brutal honesty is good?,0,[],1.0,https://www.reddit.com/r/CasualConversation/co...,all_ads,6,NaN,NaN,"{'is_prediction': False, 'options': [{'id': '5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,Internal_Use8954,NaN,[],NaN,text,t2_2smn61ln,False,F

In [6]:
columns = ['subreddit', 'author', 'selftext', 'title', 'id']

In [7]:
casual = casual[columns]

In [8]:
political = political[columns]

In [9]:
df = pd.concat([casual, political], ignore_index=True)

In [10]:
df['binary_sub'] = df['subreddit'].map({'CasualConversation':0, 'PoliticalDiscussion':1})

In [11]:
df['totaltext'] = df['selftext'] + ' ' + df['title']

In [12]:
analyzer = SentimentIntensityAnalyzer()

for count, text in enumerate(df['totaltext']):
    vs = analyzer.polarity_scores(text)
    df.loc[count, 'vader_neg'] = vs['neg']
    df.loc[count, 'vader_pos'] = vs['pos']
    df.loc[count, 'vader_neu'] = vs['neu']
    df.loc[count, 'vader_compond'] = vs['compound']

In [13]:
df

,subreddit,author,selftext,title,id,binary_sub,totaltext,vader_neg,vader_pos,vader_neu,vader_compond
0,CasualConversation,V-Jessica-V,(Name): Jessica\n(Age): 21\n(Height): 5’10 (So...,"F, 21. Looking for people to chat to :D",l7e0yt,0,(Name): Jessica\n(Age): 21\n(Height): 5’10 (So...,0.153,0.241,0.606,0.9334
1,CasualConversation,shaggy1452,I haven’t done contact spirts (or any sports b...,Did my first Brazilian Jiujitsu class today,l7dxgq,0,I haven’t done contact spirts (or any sports b...,0.063,0.146,0.791,0.9431
2,CasualConversation,old_lady_young_body,Guyssssssssssssss maybe I'm late to the game o...,Cleaning my converse,l7dku0,0,Guyssssssssssssss maybe I'm late to the game o...,0.057,0.159,0.784,0.8417
3,CasualConversation,randomusername_741,\n\n[View Poll](https://www.reddit.com/poll/l7...,Do you think brutal honesty is good?,l7dfv6,0,\n\n[View Poll](https://www.reddit.com/poll/l7...,0.253,0.377,0.370,0.2500
4,CasualConversation,Internal_Use8954,Title says it. It was Ben &amp; Jerry's Pistac...,I just tried pistachio ice cream for the first...,l7d0tf,0,Title says it. It was Ben &amp; Jerry's Pistac...,0.064,0.115,0.820,0.7401
...,...,...,...,...,...,...,...,...,...,...,...
20053,PoliticalDiscussion,JW_2,-Why is Trump unable to be attacked?\n\n-Why i...,What is the best way for Trump's opponents to ...,48u2ww,1,-Why is Trump unable to be attacked?\n\n-Why i...,0.261,0.146,0.593,-0.6767
20054,PoliticalDiscussion,toby224,It seems pretty odd that many people in this s...,The establishment will not allow a third party...,48u223,1,It seems pretty odd that many people in this s...,0.083,0.262,0.655,0.9708
20055,PoliticalDiscussion,itsasecretoeverybody,"Currently, the President of the United States ...",Should the President have the unrestricted pow...,48tydy,1,"Currently, the President of the United States ...",0.047,0.162,0.791,0.9044
20056,PoliticalDiscussion,MakeVolsGreatAgain,Coming off the great discussion we had about t...,What does 'neoconservatism' (in foreign policy...,48tx2a,1,Coming off the great discussion we had about t...,0.033,0.091,0.876,0.9072


Special characters need to be removed from selftext and title. This needs to be edited. 

In [14]:
# # https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python
# df['selftext'] = df['selftext'].map(lambda x: re.sub('http[s]?://\S+', '', x)) # removing urls

# # https://stackoverflow.com/questions/5075247/regular-expression-to-remove-line-breaks
# df['selftext'] = df['selftext'].map(lambda x: re.sub('r"(?<=[a-z])\r\n"', ' ', x)) # Removing line breaks

# # https://stackoverflow.com/questions/1219915/regex-to-remove-apostrophe
# df['selftext'] = df['selftext'].map(lambda x: re.sub("/'+/g", '', x)) # Removing apostrophes


# df['selftext'] = df['selftext'].map(lambda x: re.sub('\[(.*?)\]', ' ', x)) #removing text in brackets


# #df['selftext'] = df['selftext'].map(lambda x: re.sub('\xa0', ' ', x)) # removing xa0


# #df['selftext'] = df['selftext'].map(lambda x: re.sub('\n', ' ', x)) # removing line break


# #df['selftext'] = df['selftext'].map(lambda x: re.sub('\s[\/]?r\/[^\s]+', ' ', x)) # removing mentions of any subreddit


# #df['selftext'] = df['selftext'].map(lambda x: re.sub('http[s]?://\S+', '', x)) # removing urls

# df['selftext'] = df['selftext'].map(lambda x: re.sub("\d+", ' ', x)) # Deleting numbers

# df['selftext'] = df['selftext'].map(lambda x: re.sub("[^a-zA-Z]", ' ', x)) # Only keeping letters


# df['selftext'] = df['selftext'].map(lambda x: re.sub('[^\w\s]', ' ', x)) # Removing all punctuation 

# # https://stackoverflow.com/questions/2403122/regular-expression-to-extract-text-between-square-brackets


In [15]:
def regex_remover(column):
    
    # https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python
    df[column] = df[column].map(lambda x: re.sub(r'http[s]?://\S+', '', x)) # removing urls

    # https://stackoverflow.com/questions/5075247/regular-expression-to-remove-line-breaks
    df[column] = df[column].map(lambda x: re.sub(r"(?<=[a-z])\r\n", ' ', x)) # Removing line breaks

    # https://stackoverflow.com/questions/1219915/regex-to-remove-apostrophe
    #df[column] = df[column].map(lambda x: re.sub("/'+/g", '', x)) # Removing apostrophes

    df[column] = df[column].map(lambda x: re.sub(r'\[(.*?)\]', ' ', x)) #removing text in brackets
    
    
    df[column] = df[column].map(lambda x: re.sub(r'\d+', ' ', x)) # Deleting numbers
    
    # https://stackoverflow.com/questions/12985456/replace-all-non-alphanumeric-characters-in-a-string
    df[column] = df[column].map(lambda x: re.sub(r'[^a-zA-Z]', ' ', x)) # Only keeping letters

    #https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
    df[column] = df[column].map(lambda x: re.sub(r'[^\w\s]', ' ', x)) # Removing all punctuation 

    return df

In [16]:
df = regex_remover('selftext')

df = regex_remover('title')

In [17]:
df['totaltext'] = df['selftext'] + ' ' + df['title']

In [18]:
df

,subreddit,author,selftext,title,id,binary_sub,totaltext,vader_neg,vader_pos,vader_neu,vader_compond
0,CasualConversation,V-Jessica-V,Name Jessica Age Height Someth...,F Looking for people to chat to D,l7e0yt,0,Name Jessica Age Height Someth...,0.153,0.241,0.606,0.9334
1,CasualConversation,shaggy1452,I haven t done contact spirts or any sports b...,Did my first Brazilian Jiujitsu class today,l7dxgq,0,I haven t done contact spirts or any sports b...,0.063,0.146,0.791,0.9431
2,CasualConversation,old_lady_young_body,Guyssssssssssssss maybe I m late to the game o...,Cleaning my converse,l7dku0,0,Guyssssssssssssss maybe I m late to the game o...,0.057,0.159,0.784,0.8417
3,CasualConversation,randomusername_741,,Do you think brutal honesty is good,l7dfv6,0,Do you think brutal honesty is good,0.253,0.377,0.370,0.2500
4,CasualConversation,Internal_Use8954,Title says it It was Ben amp Jerry s Pistac...,I just tried pistachio ice cream for the first...,l7d0tf,0,Title says it It was Ben amp Jerry s Pistac...,0.064,0.115,0.820,0.7401
...,...,...,...,...,...,...,...,...,...,...,...
20053,PoliticalDiscussion,JW_2,Why is Trump unable to be attacked Why is ...,What is the best way for Trump s opponents to ...,48u2ww,1,Why is Trump unable to be attacked Why is ...,0.261,0.146,0.593,-0.6767
20054,PoliticalDiscussion,toby224,It seems pretty odd that many people in this s...,The establishment will not allow a third party...,48u223,1,It seems pretty odd that many people in this s...,0.083,0.262,0.655,0.9708
20055,PoliticalDiscussion,itsasecretoeverybody,Currently the President of the United States ...,Should the President have the unrestricted pow...,48tydy,1,Currently the President of the United States ...,0.047,0.162,0.791,0.9044
20056,PoliticalDiscussion,MakeVolsGreatAgain,Coming off the great discussion we had about t...,What does neoconservatism in foreign policy...,48tx2a,1,Coming off the great discussion we had about t...,0.033,0.091,0.876,0.9072


In [19]:
df['id'].value_counts()

4gy682    1
kg8vgv    1
4epnjc    1
7ffl18    1
ka5c88    1
         ..
kb6n4h    1
4a9361    1
ko5z26    1
4ip6q7    1
knydiv    1
Name: id, Length: 20058, dtype: int64

lemmatization

In [20]:
#https://stackoverflow.com/questions/15388831/what-are-all-possible-pos-tags-of-nltk
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

# https://stackoverflow.com/questions/10674832/count-verbs-nouns-and-other-parts-of-speech-with-pythons-nltk
def pos_counter(text):
    tok = WordPunctTokenizer()
    tagger = pos_tag(tok.tokenize(text))
    counts = Counter(tag for word, tag in tagger)
    return counts                 
    
# https://stackoverflow.com/questions/51267166/lemmatization-pandas-python                
def lemmatize_text(sentence):
    #tokenize the sentence and find the POS tag for each token
#     nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
#     #tuple of (token, wordnet_tag)
#     wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatizer = WordNetLemmatizer()
    tok = WordPunctTokenizer()
    tagger = pos_tag(tok.tokenize(sentence))
    pos_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), tagger)
    lemmas = []
    for word, tag in pos_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmas.append(word)
        else:
            #else use the tag to lemmatize the token
            lemmas.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmas)

In [21]:
df['pos_counts'] = df['totaltext'].apply(lambda x: pos_counter(x))

In [22]:
pos_list = list(df.loc[0,'pos_counts'].keys())

In [23]:
df

,subreddit,author,selftext,title,id,binary_sub,totaltext,vader_neg,vader_pos,vader_neu,vader_compond,pos_counts
0,CasualConversation,V-Jessica-V,Name Jessica Age Height Someth...,F Looking for people to chat to D,l7e0yt,0,Name Jessica Age Height Someth...,0.153,0.241,0.606,0.9334,"{'NN': 17, 'NNP': 9, 'VBG': 5, 'PRP': 13, 'VBP..."
1,CasualConversation,shaggy1452,I haven t done contact spirts or any sports b...,Did my first Brazilian Jiujitsu class today,l7dxgq,0,I haven t done contact spirts or any sports b...,0.063,0.146,0.791,0.9431,"{'PRP': 16, 'VBP': 9, 'JJ': 15, 'VBN': 2, 'NN'..."
2,CasualConversation,old_lady_young_body,Guyssssssssssssss maybe I m late to the game o...,Cleaning my converse,l7dku0,0,Guyssssssssssssss maybe I m late to the game o...,0.057,0.159,0.784,0.8417,"{'NNP': 2, 'RB': 7, 'PRP': 13, 'VBP': 7, 'JJ':..."
3,CasualConversation,randomusername_741,,Do you think brutal honesty is good,l7dfv6,0,Do you think brutal honesty is good,0.253,0.377,0.370,0.2500,"{'VBP': 1, 'PRP': 1, 'VB': 1, 'JJ': 2, 'NN': 1..."
4,CasualConversation,Internal_Use8954,Title says it It was Ben amp Jerry s Pistac...,I just tried pistachio ice cream for the first...,l7d0tf,0,Title says it It was Ben amp Jerry s Pistac...,0.064,0.115,0.820,0.7401,"{'NNP': 9, 'VBZ': 3, 'PRP': 20, 'VBD': 10, 'NN..."
...,...,...,...,...,...,...,...,...,...,...,...,...
20053,PoliticalDiscussion,JW_2,Why is Trump unable to be attacked Why is ...,What is the best way for Trump s opponents to ...,48u2ww,1,Why is Trump unable to be attacked Why is ...,0.261,0.146,0.593,-0.6767,"{'WRB': 2, 'VBZ': 4, 'NNP': 6, 'JJ': 2, 'TO': ..."
20054,PoliticalDiscussion,toby224,It seems pretty odd that many people in this s...,The establishment will not allow a third party...,48u223,1,It seems pretty odd that many people in this s...,0.083,0.262,0.655,0.9708,"{'PRP': 3, 'VBZ': 7, 'JJ': 6, 'IN': 12, 'DT': ..."
20055,PoliticalDiscussion,itsasecretoeverybody,Currently the President of the United States ...,Should the President have the unrestricted pow...,48tydy,1,Currently the President of the United States ...,0.047,0.162,0.791,0.9044,"{'RB': 4, 'DT': 15, 'NNP': 18, 'IN': 6, 'NNPS'..."
20056,PoliticalDiscussion,MakeVolsGreatAgain,Coming off the great discussion we had about t...,What does neoconservatism in foreign policy...,48tx2a,1,Coming off the great discussion we had about t...,0.033,0.091,0.876,0.9072,"{'VBG': 4, 'RP': 1, 'DT': 18, 'JJ': 24, 'NN': ..."


In [24]:
#df['pos_counts'].map(lambda x: x.get('NN'))
for pos in pos_list:
    total = df['pos_counts'].map(lambda x: sum(x.values()))
    df[pos] = df['pos_counts'].map(lambda x: x.get(pos))
    df[pos] = df[pos].fillna(0)
    df[pos] = df[pos]/total
    df[pos] = df[pos].fillna(0)

In [25]:
df.isnull().sum()

subreddit        0
author           0
selftext         0
title            0
id               0
binary_sub       0
totaltext        0
vader_neg        0
vader_pos        0
vader_neu        0
vader_compond    0
pos_counts       0
NN               0
NNP              0
VBG              0
PRP              0
VBP              0
TO               0
VB               0
IN               0
CC               0
JJ               0
NNS              0
DT               0
RB               0
RBR              0
PRP$             0
VBN              0
MD               0
dtype: int64

In [26]:
df

,subreddit,author,selftext,title,id,binary_sub,totaltext,vader_neg,vader_pos,vader_neu,vader_compond,pos_counts,NN,NNP,VBG,PRP,VBP,TO,VB,IN,CC,JJ,NNS,DT,RB,RBR,PRP$,VBN,MD
0,CasualConversation,V-Jessica-V,Name Jessica Age Height Someth...,F Looking for people to chat to D,l7e0yt,0,Name Jessica Age Height Someth...,0.153,0.241,0.606,0.9334,"{'NN': 17, 'NNP': 9, 'VBG': 5, 'PRP': 13, 'VBP...",0.130769,0.069231,0.038462,0.100000,0.092308,0.038462,0.061538,0.107692,0.046154,0.100000,0.084615,0.030769,0.053846,0.007692,0.015385,0.015385,0.007692
1,CasualConversation,shaggy1452,I haven t done contact spirts or any sports b...,Did my first Brazilian Jiujitsu class today,l7dxgq,0,I haven t done contact spirts or any sports b...,0.063,0.146,0.791,0.9431,"{'PRP': 16, 'VBP': 9, 'JJ': 15, 'VBN': 2, 'NN'...",0.157895,0.011696,0.046784,0.093567,0.052632,0.040936,0.052632,0.111111,0.029240,0.087719,0.035088,0.087719,0.052632,0.000000,0.023392,0.011696,0.011696
2,CasualConversation,old_lady_young_body,Guyssssssssssssss maybe I m late to the game o...,Cleaning my converse,l7dku0,0,Guyssssssssssssss maybe I m late to the game o...,0.057,0.159,0.784,0.8417,"{'NNP': 2, 'RB': 7, 'PRP': 13, 'VBP': 7, 'JJ':...",0.102564,0.025641,0.012821,0.166667,0.089744,0.025641,0.025641,0.076923,0.051282,0.089744,0.038462,0.064103,0.089744,0.000000,0.025641,0.012821,0.012821
3,CasualConversation,randomusername_741,,Do you think brutal honesty is good,l7dfv6,0,Do you think brutal honesty is good,0.253,0.377,0.370,0.2500,"{'VBP': 1, 'PRP': 1, 'VB': 1, 'JJ': 2, 'NN': 1...",0.142857,0.000000,0.000000,0.142857,0.142857,0.000000,0.142857,0.000000,0.000000,0.285714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,CasualConversation,Internal_Use8954,Title says it It was Ben amp Jerry s Pistac...,I just tried pistachio ice cream for the first...,l7d0tf,0,Title says it It was Ben amp Jerry s Pistac...,0.064,0.115,0.820,0.7401,"{'NNP': 9, 'VBZ': 3, 'PRP': 20, 'VBD': 10, 'NN...",0.154362,0.060403,0.013423,0.134228,0.040268,0.020134,0.053691,0.087248,0.040268,0.073826,0.033557,0.060403,0.080537,0.000000,0.020134,0.000000,0.013423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20053,PoliticalDiscussion,JW_2,Why is Trump unable to be attacked Why is ...,What is the best way for Trump s opponents to ...,48u2ww,1,Why is Trump unable to be attacked Why is ...,0.261,0.146,0.593,-0.6767,"{'WRB': 2, 'VBZ': 4, 'NNP': 6, 'JJ': 2, 'TO': ...",0.139535,0.139535,0.000000,0.046512,0.023256,0.069767,0.069767,0.046512,0.023256,0.046512,0.069767,0.069767,0.000000,0.000000,0.000000,0.023256,0.000000
20054,PoliticalDiscussion,toby224,It seems pretty odd that many people in this s...,The establishment will not allow a third party...,48u223,1,It seems pretty odd that many people in this s...,0.083,0.262,0.655,0.9708,"{'PRP': 3, 'VBZ': 7, 'JJ': 6, 'IN': 12, 'DT': ...",0.128713,0.128713,0.039604,0.029703,0.039604,0.019802,0.019802,0.118812,0.019802,0.059406,0.049505,0.148515,0.039604,0.000000,0.009901,0.039604,0.009901
20055,PoliticalDiscussion,itsasecretoeverybody,Currently the President of the United States ...,Should the President have the unrestricted pow...,48tydy,1,Currently the President of the United States ...,0.047,0.162,0.791,0.9044,"{'RB': 4, 'DT': 15, 'NNP': 18, 'IN': 6, 'NNPS'...",0.110169,0.152542,0.000000,0.016949,0.042373,0.067797,0.101695,0.050847,0.042373,0.059322,0.059322,0.127119,0.033898,0.000000,0.000000,0.033898,0.042373
20056,PoliticalDiscussion,MakeVolsGreatAgain,Coming off the great discussion we had about t...,What does neoconservatism in foreign policy...,48tx2a,1,Coming off the great discussion we had about t...,0.033,0.091,0.876,0.9072,"{'VBG': 4, 'RP': 1, 'DT': 18, 'JJ': 24, 'NN': ...",0.167421,0.081448,0.018100,0.054299,0.031674,0.031674,0.031674,0.144796,0.040724,0.108597,0.040724,0.081448,0.031674,0.000000,0.000000,0.013575,0.004525


In [27]:
# class balance
df['binary_sub'].value_counts(normalize=True)

0    0.501446
1    0.498554
Name: binary_sub, dtype: float64

In [28]:
df['lemma_text'] = df['selftext'].apply(lambda x: lemmatize_text(x))

In [29]:
df['lemma_title'] = df['title'].apply(lambda x: lemmatize_text(x))

In [30]:
df['lemma_totaltext'] = df['lemma_title'] + ' ' + df['lemma_text']

In [31]:
tvec = TfidfVectorizer(stop_words='english', min_df=3)
x = tvec.fit_transform(df['totaltext'])

In [32]:
# df_1 = pd.DataFrame(tvec.transform(df['totaltext']).todense(),
#                    columns=tvec.get_feature_names())

In [33]:
# df_1 = df_1.T

In [34]:
# df_1['total_tfidf_score'] = df_1.sum(axis=1)

In [35]:
# df_1

In [36]:
#df_1[df_1['sparse'] < 0.5]

In [37]:
#df['totaltext']

In [38]:
# grouped = df.groupby('binary_sub')

In [39]:
# features = CountVectorizer(stop_words='english', analyzer='word')

In [40]:
#political.to_csv('./Data/political_clean.csv', index=False)

In [41]:
#casual.to_csv('./Data/casual_clean.csv', index=False)

In [42]:
df = df.sample(n=len(df), replace = False).reset_index(drop=True)

In [43]:
df = df[['binary_sub', 'NN', 'NNP', 'VBG', 'PRP', 'VBP', 'TO', 'VB','IN', 'CC',
        'JJ', 'NNS', 'DT', 'RB', 'RBR', 'PRP$', 'VBN', 'MD', 'vader_neg', 
         'vader_pos', 'vader_neu','vader_compond', 'lemma_totaltext']]

In [44]:
df.to_csv('./Data/casual_political.csv', index=False)